In [2]:
from vqvae3d_monai import VQVAE

import os
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

from dipy.io.image import load_nifti
from dipy.align.reslice import reslice
from scipy.ndimage import affine_transform

strategy = tf.distribute.MirroredStrategy()
num_gpus = strategy.num_replicas_in_sync
print(f'Number of devices: {num_gpus}')
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
print(tf.config.list_logical_devices())

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1
[]
[LogicalDevice(name='/device:CPU:0', device_type='CPU')]


2023-10-16 11:12:52.194131: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /N/soft/rhel7/deeplearning/Python-3.10.5/libcutensor/lib/11:/N/soft/rhel7/deeplearning/Python-3.10.5/libcusparse_lt/lib64:/N/soft/rhel7/deeplearning/Python-3.10.5/cuda/lib64:/N/soft/rhel7/deeplearning/Python-3.10.5/cuda/lib:/N/soft/rhel7/deeplearning/Python-3.10.5/:/N/soft/rhel7/openmpi/intel/4.0.1/lib:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/compiler/lib/intel64:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/ipp/lib/intel64:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/compiler/lib/intel64_lin:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/mkl/lib/intel64_lin:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/tbb/lib/intel64/gcc4.7:/N/soft/rhel7/intel/19.

In [3]:
def get_dataset_list(dataset):
    dataset_list = []
    
    if dataset == 'CC':
        files = ['./dataset_cc359.txt']
    elif dataset == 'NFBS':
        files = ['./dataset.txt']
    elif dataset == 'both':
        files = ['./dataset.txt']
        files.append('./dataset_cc359.txt')

    for file in files:
        with open(file, 'r') as f:
            lin = f.readline()
            while(lin):
                dataset_list.append(lin[:-1])
                lin = f.readline()
    print('Total Images in dataset: ', len(dataset_list))
    return dataset_list

def datasetHelperFunc(path):
    transform_vol, mask = None, None
    if isinstance(path, bytes):
        path = str(path.decode('utf-8'))

    if 'CC' in path:
        vol, affine, voxsize = load_nifti(
            '/N/project/grg_data/data/skullstripping_datasets/CC359/Original/'+path, return_voxsize=True)
        mask, _ = load_nifti(
            '/N/project/grg_data/data/skullstripping_datasets/CC359/STAPLE/'+path[:-7]+'_staple.nii.gz')
        mask[mask < 1] = 0  # Values <1 in the mask is background
        vol = vol*mask
    else:
        #vol, affine, voxsize = load_nifti(str(path.decode('utf-8')), return_voxsize=True)
        vol, affine, voxsize = load_nifti(path, return_voxsize=True)
        mask, _ = load_nifti(path[:-7]+'mask.nii.gz')
        mask[mask < 1] = 0  # Values <1 in the mask is background
        vol = vol*mask

    if mask is not None:
        mask, _ = transform_img(mask, affine, voxsize)
        # Handling negative pixels, occurred as a result of preprocessing
        mask[mask < 0] *= -1
        mask = np.expand_dims(mask, -1)
    transform_vol, _ = transform_img(vol, affine, voxsize)
    # Handling negative pixels, occurred as a result of preprocessing
    transform_vol[transform_vol < 0] *= -1
    transform_vol = (transform_vol-np.min(transform_vol)) / \
        (np.max(transform_vol)-np.min(transform_vol))
    transform_vol = np.expand_dims(transform_vol, -1)
    return tf.convert_to_tensor(transform_vol, tf.float32), tf.convert_to_tensor(mask, tf.float32)


dataset_list = get_dataset_list('both')

bs = 48
suffix = 'B48-both'
test_size = len(dataset_list) - (len(dataset_list)//bs)*bs

lis = dataset_list[-test_size:]
dataset = tf.data.Dataset.from_tensor_slices(lis)

dataset = dataset.map(lambda x: tf.numpy_function(func=datasetHelperFunc, inp=[x], Tout=[tf.float32, tf.float32]),
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)

test_dataset = dataset.batch(bs).prefetch(tf.data.experimental.AUTOTUNE).take(1)



Total Images in dataset:  484


In [4]:
model = VQVAE(
            in_channels=1,
            out_channels=1,
            num_channels=(32, 64, 128),
            num_res_channels=(32, 64, 128),
            num_res_layers=3,
            downsample_parameters=((2, 4, 1, 'same'), (2, 4, 1, 'same'), (2, 4, 1, 'same')),
            upsample_parameters=((2, 4, 1, 'same', 0), (2, 4, 1, 'same', 0), (2, 4, 1, 'same')),
            num_embeddings=256,
            embedding_dim=32,
            num_gpus=float(num_gpus),
            kernel_resize=False)

test_epoch = 64
model.load_weights(os.path.join('./checkpoints-vqvae-monai-scaled-128', suffix, str(test_epoch)+'.ckpt'))

In [5]:
for x, _ in test_dataset:
    np.save(f'./reconst_scaled_vqvae3d_monai/original-{suffix}.npy', x.numpy())
    reconst = model(x)
    loss = tf.reduce_mean((reconst-x)**2)
    print(f'Test Loss is {loss}')
    np.save(f'./reconst_scaled_vqvae3d_monai/reconst3d-{suffix}-epoch{test_epoch}.npy', reconst.numpy())
    break

2023-10-16 11:12:55.222790: W tensorflow/core/framework/op_kernel.cc:1768] RESOURCE_EXHAUSTED: MemoryError: 
Traceback (most recent call last):

  File "/N/u/aajais/Carbonate/.local/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/N/u/aajais/Carbonate/.local/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/tmp/ipykernel_6873/3510424823.py", line 27, in datasetHelperFunc
    vol, affine, voxsize = load_nifti(

  File "/N/u/aajais/Carbonate/.local/lib/python3.10/site-packages/dipy/io/image.py", line 67, in load_nifti
    data = np.asanyarray(img.dataobj) if as_ndarray else img.dataobj

  File "/N/soft/rhel7/deeplearning/Python-3.10.5/lib/python3.10/site-packages/nibabel/arrayproxy.py", line 391, in __array__
    arr = self._get_scaled(dtype=dtype, slicer=())

  File "/N/soft/rhel7/deeplearning/Python-3.10.5/lib/python3.10/site-packa

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/N/soft/rhel7/deeplearning/Python-3.10.5/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
  File "/tmp/ipykernel_6873/631462050.py", line 1, in <cell line: 1>
  File "/N/u/aajais/Carbonate/.local/lib/python3.10/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 766, in __next__
  File "/N/u/aajais/Carbonate/.local/lib/python3.10/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 749, in _next_internal
  File "/N/u/aajais/Carbonate/.local/lib/python3.10/site-packages/tensorflow/python/ops/gen_dataset_ops.py", line 3017, in iterator_get_next
  File "/N/u/aajais/Carbonate/.local/lib/python3.10/site-packages/tensorflow/python/framework/ops.py", line 7209, in raise_from_not_ok_status
tensorflow.python.framework.errors_impl.ResourceExhaustedError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} MemoryError: 
Traceback (most

In [ ]:
for x, _ in test_dataset:
    print(x.numpy().shape)
    break

In [ ]:
from vqvae3d_monai import VQVAE